Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [101]:
import numpy as np
rng = np.random.default_rng(seed=42)

In [102]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2

In [103]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [ ]:
WEIGHTS

In [105]:
CONSTRAINTS

array([[ 67, 231],
       [165, 168],
       [450, 247]])

In [106]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) <0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

In [ ]:
solution

In [124]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

MAXITER = 2000

In [109]:
#variable to limit the number of attempts to find a valid configuration, used in both functions for the single knapsack and for all knapsacks
#avoid this value: definitely too long wait (with 1000 problem 3 took around 30 minutes to complete, so I'd use it just for problem 1)
approximation_limit = 10000

In [110]:
approximation_limit = 1000

In [111]:
approximation_limit = 100

In [112]:
def checkKnapsack(solution, knapsack):
    #checks if the solution is valid
    return np.all(WEIGHTS[solution[knapsack]].sum(axis=0) <= CONSTRAINTS[knapsack])

def fillKnapsack(knapsacks, numKnapsack, mask):
    #process: get all the values that can be chosen, get a random set of items to put in the knapsack
    #         and then check if the solution is valid

    for j in range(approximation_limit):
        
        #checking if there are still items to choose from
        if(np.sum(~mask) == 0):
            break
        
        #choosing knapsack items
        chosen_items = np.random.choice(np.where(~mask)[0], size=np.random.randint(1, np.sum(~mask)+1), replace=False)

        #creating the mask for the knapsack to be filled
        new_mask = np.zeros_like(mask, dtype=bool)
        new_mask[chosen_items] = np.True_

        #creating a copy of the knapsacks to test the new configuration
        knapsackCopy = knapsacks.copy()
        knapsackCopy[numKnapsack] |= new_mask

        #checking if the knapsack is valid
        if checkKnapsack(knapsackCopy, numKnapsack):
            return new_mask
        

    return np.zeros_like(mask, dtype=bool)
        
def calculateValue(solution):
    total_value = 0
    for k in range(NUM_KNAPSACKS):
        total_value += VALUES[solution[k]].sum()
    return total_value

def createSolution():

    #Variables used for tracking an approximate solution
    best_solution = None
    best_solution_value = 0

    for k in range(approximation_limit):  # Outer loop to restart the for loop

        #new solution initialization
        solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
        mask = np.zeros(NUM_ITEMS, dtype=bool)

        #loop to fill knapsacks
        for j in range(NUM_KNAPSACKS):

            # Filling the j-th knapsack
            solution[j] = fillKnapsack(solution, j, mask)

            # Condition to continue the loop, if the knapsack obtained is not empty probably there's still objects for other knapsacks
            if solution[j].sum() != 0:  
                mask |= solution[j]
                if(j == NUM_KNAPSACKS - 1):
                    
                    #checking if the solution is better than the best one found yet
                    value = calculateValue(solution)
                    if best_solution is None or value > best_solution_value:
                        best_solution = solution.copy()
                        best_solution_value = value
                continue
            
            value = calculateValue(solution)
            if best_solution is None or value > best_solution_value:
                best_solution = solution.copy()
                best_solution_value = value
            
            # Exit the for loop to restart
            break  
    # Return the best solution found
    print("Best solution with value ", best_solution_value)
    return best_solution  


In [113]:
def validate(solution):
    if not np.all(solution.sum(axis=0) <= 1):
        return False #if an item is in more than one knapsack exit
    
    for nap in range(NUM_KNAPSACKS):
        weights = WEIGHTS[solution[nap]].sum(axis=0)
        if np.any(weights > CONSTRAINTS[nap]):
            return False #if invalid exit
    return True

In [114]:
def cost(solution):
    all_knapsacks = np.any(solution, axis=0)
    all_cost = VALUES[all_knapsacks].sum()
    return all_cost

In [115]:
def tweak(solution, p = 0.4):
    new_solution = solution.copy()
    if np.random.random() < p:
        # Swap two items between two knapsacks
        knapsack1, knapsack2 = np.random.choice(NUM_KNAPSACKS, size=2, replace=False)
        item1 = np.random.randint(0, NUM_ITEMS)
        item2 = np.random.randint(0, NUM_ITEMS)
        new_solution[knapsack1, item1], new_solution[knapsack2, item2] = (
            new_solution[knapsack2, item2],
            new_solution[knapsack1, item1],
        )
    else:
        # Add or remove an item from a knapsack
        knapsack = np.random.randint(0, NUM_KNAPSACKS)
        item = np.random.randint(0, NUM_ITEMS)
        new_solution[knapsack, item] = not new_solution[knapsack, item]
    
    return new_solution

In [116]:
def hill_climbing(solution):
    current_cost = cost(solution)

    best_solution = solution
    best_cost = current_cost

    p = 0.2
    for i in range(MAXITER):
        new_solution = tweak(solution)
        new_cost = cost(new_solution)
        if new_cost >= current_cost and validate(new_solution):
            solution = new_solution
            current_cost = new_cost
            if current_cost > best_cost:
                print(f"Iteration {i}: current cost = {best_cost}, new cost = {current_cost}")
                best_cost = current_cost
                best_solution = solution
        elif validate(new_solution) and np.random.random() < p:
            p = p * 0.99
            solution = new_solution
            current_cost = new_cost
            
    return best_solution

In [ ]:

solution = createSolution()
solution = hill_climbing(solution)
solution

In [118]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

np.True_

## TEST PROBLEMS

In [119]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

In [120]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [121]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)